In [1]:
##DEPENDENCIES
import pandas as pd
import datetime
from datetime import timedelta, date
import yfinance as yf
import numpy
import yahoofinancials
from yahoofinancials import YahooFinancials

#define quarter yyyyQq
q = '2020Q1'

In [2]:
#import earnings dates data ***check file***
sp_earnings = pd.read_csv('resources/'+q+'_sp_earnings.csv', index_col=False)
del sp_earnings['Unnamed: 0']
sp_earnings.head()

,Symbol,2020Q1 Time,2020Q1 Date
0,A,TAS,2020-02-18
1,AAL,TAS,2020-01-23
2,AAP,TAS,2020-02-18
3,AAPL,TAS,2020-01-28
4,ABBV,TAS,2020-02-07


In [3]:
## define function to collect previous, earnings, and next day ohlc quotes given the symbol and earnings date
def ohlc(sym, earn_date):
        yf_sym = YahooFinancials([sym])
        prev = []
        earn = []
        nxt = []
        try:    
            ##PREVIOUS DAY DATA
            #calculating the previous day
            theday = datetime.date(*map(int, earn_date.split('-')))
            prevday = theday - datetime.timedelta(days=1)
            prevday = prevday.strftime('%Y-%m-%d')
            prev_date = prevday
            print(sym+' '+earn_date)
            #data for previous day
            data = yf_sym.get_historical_price_data(start_date= prevday, 
                                                  end_date= earn_date, 
                                                  time_interval='daily')
            #if market closed on previous day this keeps trying one day previous until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, prevday.split('-')))
                earn_prev = prevday
                prevday = theday - datetime.timedelta(days=1)
                prevday = prevday.strftime('%Y-%m-%d')
                prev_date = prevday
                data = yf_sym.get_historical_price_data(start_date= prevday, 
                                                  end_date= earn_prev, 
                                                  time_interval='daily') 
            #previous day OHLC
            prev_open = data[sym]['prices'][0]['open']
            prev_low = data[sym]['prices'][0]['low']
            prev_high = data[sym]['prices'][0]['high']            
            prev_close = data[sym]['prices'][0]['close']
            prev = [prev_date, prev_open, prev_low, prev_high, prev_close]

            ##EARNINGS DATE DATA
            #calculating next day
            theday = datetime.date(*map(int, earn_date.split('-')))
            nextday = theday + datetime.timedelta(days=1)
            nextday = nextday.strftime('%Y-%m-%d')
            next_date = nextday
            #data for next day
            data = yf_sym.get_historical_price_data(start_date= earn_date, 
                                        end_date= nextday, 
                                        time_interval='daily')
            #if market closed on next day this keeps trying one day ahead until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, nextday.split('-')))
                earn_next = nextday
                nextday = theday + datetime.timedelta(days=1)
                nextday = nextday.strftime('%Y-%m-%d')
                data = yf_sym.get_historical_price_data(start_date= earn_next, 
                                                  end_date= nextday, 
                                                  time_interval='daily') 
            #earnings day OHLC
            earn_open = data[sym]['prices'][0]['open']
            earn_low = data[sym]['prices'][0]['low']
            earn_high = data[sym]['prices'][0]['high']            
            earn_close = data[sym]['prices'][0]['close']
            earn = [earn_date, earn_open, earn_low, earn_high, earn_close]
            
            ##NEXT DATE DATA
            #calculating day after next
            theday = datetime.date(*map(int, next_date.split('-')))
            nextday = theday + datetime.timedelta(days=1)
            nextday = nextday.strftime('%Y-%m-%d')
            #data for next day
            data = yf_sym.get_historical_price_data(start_date= next_date, 
                                        end_date= nextday, 
                                        time_interval='daily')
            #if market closed on day after next this keeps trying one day ahead until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, nextday.split('-')))
                earn_next = nextday
                next_date = nextday
                nextday = theday + datetime.timedelta(days=1)
                nextday = nextday.strftime('%Y-%m-%d')
                data = yf_sym.get_historical_price_data(start_date= earn_next, 
                                                  end_date= nextday, 
                                                  time_interval='daily') 
            #next day OHLC
            next_open = data[sym]['prices'][0]['open']
            next_low = data[sym]['prices'][0]['low']
            next_high = data[sym]['prices'][0]['high']            
            next_close = data[sym]['prices'][0]['close']
            nxt.append(next_date) 
            nxt.append(next_open)
            nxt.append(next_low)
            nxt.append(next_high)
            nxt.append(next_close)
            
        except:
            print(sym+' NaN')
            prev_date = 'NaN'
            prev_open = 'NaN'
            prev_low = 'NaN'
            prev_high = 'NaN'            
            prev_close = 'NaN'
            earn_date = 'NaN'
            earn_open = 'NaN'
            earn_low = 'NaN'
            earn_high = 'NaN'       
            earn_close = 'NaN'
            next_date = 'NaN'
            next_open = 'NaN'
            next_low = 'NaN'
            next_high = 'NaN'       
            next_close = 'NaN' 

        quotes = {'Symbol' : sym,
                q+' Prev Date' : prev_date,
                q+' Prev Open' : prev_open, 
                q+' Prev High' : prev_high, 
                q+' Prev Low' : prev_low, 
                q+' Prev Close' : prev_close, 
                q+' Earn Date' : earn_date, 
                q+' Earn Open' : earn_open, 
                q+' Earn High' : earn_high, 
                q+' Earn Low' : earn_low, 
                q+' Earn Close' : earn_close,
                q+' Next Date' : next_date, 
                q+' Next Open' : next_open,
                q+' Next High' : next_high,
                q+' Next Low' : next_low, 
                q+' Next Close' : next_close}
        return quotes;

In [4]:
#collect ohlc data for every symbol and add each dictionary to a list
symbols = list(sp_earnings['Symbol'])
date_prices = []

for row in sp_earnings.iterrows():
    quotes = ohlc(row[1]['Symbol'], row[1][q+' Date'])
    date_prices.append(quotes)

date_prices    

A 2020-02-18
AAL 2020-01-23
AAP 2020-02-18
AAPL 2020-01-28
ABBV 2020-02-07
ABC 2020-01-30
ABMD 2020-02-06
ABT 2020-01-22
ACN 2020-03-19
ADBE 2020-03-12
ADI 2020-02-19
ADM 2020-01-29
ADP 2020-01-29
ADS 2020-01-30
ADSK 2020-02-27
AEE 2020-02-26
AEP 2020-02-20
AES 2020-02-28
AFL 2020-02-04
AGN 2020-02-10
AIG 2020-02-13
AIV 2020-01-30
AIZ 2020-02-10
AJG 2020-01-30
AKAM 2020-02-11
ALB 2020-02-19
ALGN 2020-01-29
ALK 2020-01-28
ALL 2020-02-04
ALLE 2020-02-18
ALXN 2020-01-30
AMAT 2020-02-12
AMCR NaN
AMD 2020-01-28
AME 2020-02-05
AMGN 2020-01-30
AMP 2020-01-29
AMT 2020-02-25
AMZN 2020-01-30
ANET 2020-02-13
ANSS 2020-02-26
ANTM 2020-01-29
AON 2020-01-31
AOS 2020-01-28
APA 2020-02-26
APD 2020-01-24
APH 2020-01-22
APTV 2020-01-30
ARE 2020-02-03
ATO 2020-02-04
ATVI 2020-02-06
AVB 2020-02-05
AVGO 2020-03-12
AVY 2020-01-29
AWK 2020-02-18
AXP 2020-01-24
AZO 2020-03-03
BA 2020-01-29
BAC 2020-01-15
BAX 2020-03-17
BBY 2020-02-27
BDX 2020-02-06
BEN 2020-01-30
BF.B NaN
BIIB 2020-01-30
BK 2020-01-16
BKNG 20

[{'Symbol': 'A',
  '2020Q1 Prev Date': '2020-02-14',
  '2020Q1 Prev Open': 85.72000122070312,
  '2020Q1 Prev High': 85.86000061035156,
  '2020Q1 Prev Low': 85.12000274658203,
  '2020Q1 Prev Close': 85.81999969482422,
  '2020Q1 Earn Date': '2020-02-18',
  '2020Q1 Earn Open': 85.66000366210938,
  '2020Q1 Earn High': 86.08000183105469,
  '2020Q1 Earn Low': 84.52999877929688,
  '2020Q1 Earn Close': 84.79000091552734,
  '2020Q1 Next Date': '2020-02-19',
  '2020Q1 Next Open': 84.69999694824219,
  '2020Q1 Next High': 85.83999633789062,
  '2020Q1 Next Low': 83.44999694824219,
  '2020Q1 Next Close': 85.3499984741211},
 {'Symbol': 'AAL',
  '2020Q1 Prev Date': '2020-01-22',
  '2020Q1 Prev Open': 27.510000228881836,
  '2020Q1 Prev High': 27.90999984741211,
  '2020Q1 Prev Low': 27.299999237060547,
  '2020Q1 Prev Close': 27.31999969482422,
  '2020Q1 Earn Date': '2020-01-23',
  '2020Q1 Earn Open': 27.389999389648438,
  '2020Q1 Earn High': 28.989999771118164,
  '2020Q1 Earn Low': 26.040000915527344,
 

In [5]:
#create a dataframe from the ohlc list
dp_df = pd.DataFrame(date_prices)
dp_df = dp_df[['Symbol',
  q+' Prev Date',
  q+' Prev Open',
  q+' Prev High',
  q+' Prev Low',
  q+' Prev Close',
  q+' Earn Date',
  q+' Earn Open',
  q+' Earn High',
  q+' Earn Low',
  q+' Earn Close',
  q+' Next Date',
  q+' Next Open',
  q+' Next High',
  q+' Next Low',
  q+' Next Close']]
dp_df.head()

,Symbol,2020Q1 Prev Date,2020Q1 Prev Open,2020Q1 Prev High,2020Q1 Prev Low,2020Q1 Prev Close,2020Q1 Earn Date,2020Q1 Earn Open,2020Q1 Earn High,2020Q1 Earn Low,2020Q1 Earn Close,2020Q1 Next Date,2020Q1 Next Open,2020Q1 Next High,2020Q1 Next Low,2020Q1 Next Close
0,A,2020-02-14,85.72,85.86,85.12,85.82,2020-02-18,85.66,86.08,84.53,84.79,2020-02-19,84.7,85.84,83.45,85.35
1,AAL,2020-01-22,27.51,27.91,27.3,27.32,2020-01-23,27.39,28.99,26.04,28.8,2020-01-24,28.85,28.86,27.01,27.64
2,AAP,2020-02-14,134.75,134.75,131.56,133.59,2020-02-18,145.39,150.66,140.76,141.81,2020-02-19,141.51,142.42,137.74,140.12
3,AAPL,2020-01-27,310.06,311.77,304.88,308.95,2020-01-28,312.6,318.4,312.19,317.69,2020-01-29,324.45,327.85,321.38,324.34
4,ABBV,2020-02-06,87.89,87.96,86.22,87.18,2020-02-07,89.74,92.98,88.88,92.29,2020-02-10,92.36,95.5,92.11,94.75


In [6]:
#calculate the percent changes following earnings and added to dataframe
bopen_change = []
aclose_change = []
for row in dp_df.iterrows():
    pr_cl = float(row[1][q+' Prev Close'])
    e_op = float(row[1][q+' Earn Open'])
    e_cl = float(row[1][q+' Earn Close'])
    nx_op = float(row[1][q+' Next Open'])
    bo_change = ((e_op - pr_cl)/(pr_cl))*100
    bo_change = round(bo_change, 2)
    ac_change = ((nx_op - e_cl)/(e_cl))*100
    ac_change = round(ac_change, 2)
    bopen_change.append(bo_change)
    aclose_change.append(ac_change)

dp_df[q+' Before Open %Change'] = bopen_change
dp_df[q+' After Close %Change'] = aclose_change
dp_df


,Symbol,2020Q1 Prev Date,2020Q1 Prev Open,2020Q1 Prev High,2020Q1 Prev Low,2020Q1 Prev Close,2020Q1 Earn Date,2020Q1 Earn Open,2020Q1 Earn High,2020Q1 Earn Low,2020Q1 Earn Close,2020Q1 Next Date,2020Q1 Next Open,2020Q1 Next High,2020Q1 Next Low,2020Q1 Next Close,2020Q1 Before Open %Change,2020Q1 After Close %Change
0,A,2020-02-14,85.72,85.86,85.12,85.82,2020-02-18,85.66,86.08,84.53,84.79,2020-02-19,84.7,85.84,83.45,85.35,-0.19,-0.11
1,AAL,2020-01-22,27.51,27.91,27.3,27.32,2020-01-23,27.39,28.99,26.04,28.8,2020-01-24,28.85,28.86,27.01,27.64,0.26,0.17
2,AAP,2020-02-14,134.75,134.75,131.56,133.59,2020-02-18,145.39,150.66,140.76,141.81,2020-02-19,141.51,142.42,137.74,140.12,8.83,-0.21
3,AAPL,2020-01-27,310.06,311.77,304.88,308.95,2020-01-28,312.6,318.4,312.19,317.69,2020-01-29,324.45,327.85,321.38,324.34,1.18,2.13
4,ABBV,2020-02-06,87.89,87.96,86.22,87.18,2020-02-07,89.74,92.98,88.88,92.29,2020-02-10,92.36,95.5,92.11,94.75,2.94,0.08
5,ABC,2020-01-29,89.49,90.85,89.2,89.86,2020-01-30,89.33,92.36,86.9,88.88,2020-01-31,89.29,89.29,85.27,85.56,-0.59,0.46
6,ABMD,2020-02-05,192.74,194.33,188.62,190.73,2020-02-06,175.41,199.26,171.15,196.01,2020-02-07,193.92,194,184,186.91,-8.03,-1.07
7,ABT,2020-01-21,88.86,90.04,88.61,89.73,2020-01-22,90.78,92.45,90.03,91.86,2020-01-23,91.81,91.85,90.16,90.73,1.17,-0.05
8,ACN,2020-03-18,151.57,159.8,142,151.15,2020-03-19,151.36,177.8,151.36,158.5,2020-03-20,157.53,159.47,147.42,149.94,0.14,-0.61
9,ADBE,2020-03-11,324.32,326.36,309,315.23,2020-03-12,290.41,310.13,283,285,2020-03-13,312.88,336,298,335.5,-7.87,9.78


In [7]:
#update sp_earnings.csv file
sp_merged = pd.merge(sp_earnings, dp_df, how='outer', on=['Symbol'])
sp_merged.to_csv('resources/'+q+'_sp_earnings.csv')
sp_merged.head()

,Symbol,2020Q1 Time,2020Q1 Date,2020Q1 Prev Date,2020Q1 Prev Open,2020Q1 Prev High,2020Q1 Prev Low,2020Q1 Prev Close,2020Q1 Earn Date,2020Q1 Earn Open,2020Q1 Earn High,2020Q1 Earn Low,2020Q1 Earn Close,2020Q1 Next Date,2020Q1 Next Open,2020Q1 Next High,2020Q1 Next Low,2020Q1 Next Close,2020Q1 Before Open %Change,2020Q1 After Close %Change
0,A,TAS,2020-02-18,2020-02-14,85.72,85.86,85.12,85.82,2020-02-18,85.66,86.08,84.53,84.79,2020-02-19,84.7,85.84,83.45,85.35,-0.19,-0.11
1,AAL,TAS,2020-01-23,2020-01-22,27.51,27.91,27.3,27.32,2020-01-23,27.39,28.99,26.04,28.8,2020-01-24,28.85,28.86,27.01,27.64,0.26,0.17
2,AAP,TAS,2020-02-18,2020-02-14,134.75,134.75,131.56,133.59,2020-02-18,145.39,150.66,140.76,141.81,2020-02-19,141.51,142.42,137.74,140.12,8.83,-0.21
3,AAPL,TAS,2020-01-28,2020-01-27,310.06,311.77,304.88,308.95,2020-01-28,312.6,318.4,312.19,317.69,2020-01-29,324.45,327.85,321.38,324.34,1.18,2.13
4,ABBV,TAS,2020-02-07,2020-02-06,87.89,87.96,86.22,87.18,2020-02-07,89.74,92.98,88.88,92.29,2020-02-10,92.36,95.5,92.11,94.75,2.94,0.08
